#Create a Payment Status dimension

In [0]:
from delta.tables import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [0]:
# Azure Blob Storage details
storage_account_name = "<storage-account-name>"
storage_account_key = "<storage-account-access-key>"
container_name = "deltalake"
file_path1 = "bronze/Trip_Transactions"
file_path2 = "bronze/Rewards_Points"

In [0]:
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

In [0]:
df1=spark.read.load(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path1}")

In [0]:
df1.show()

+-------+--------------------+--------------------+---------+-----------+------------------------+-----------+---------------------+--------------+-----------------------------+----------------+--------------------------+-------------------+--------------+----------+-----------+---------------------+--------------------+--------------------+-----------+-------------+
|trip_id|trip_start_timestamp|  trip_end_timestamp|driver_id|driver_name|source_location_address1|source_city|source_province_state|source_country|destination_location_address1|destination_city|destination_province_state|destination_country|total_distance|total_fare|trip_status|delay_start_time_mins|      payment_method|      payment_Status|customer_id|customer_name|
+-------+--------------------+--------------------+---------+-----------+------------------------+-----------+---------------------+--------------+-----------------------------+----------------+--------------------------+-------------------+--------------+----

In [0]:
df2=df1.select("trip_id","payment_method","payment_Status","trip_start_timestamp")

In [0]:
df2=df2.withColumn("Due_Date",F.expr("to_date(trip_start_timestamp)+7"))

In [0]:
file_path7 = "silver_Zone/Payment_Status_Dimension"

In [0]:
df2.repartition(1).write.save(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path7}")